#### Parte 3: Arquitectura de una CNN

Una arquitectura típica de CNN consiste en capas alternadas de convolución y pooling, seguidas de capas completamente conectadas:




- Una capa convolucional con 32 filtros, tamaño de kernel 3x3 y activación ReLU.
- Una capa de max pooling con tamaño de pool 2x2.
- Una segunda capa convolucional con 64 filtros, tamaño de kernel 3x3 y activación ReLU.
- Una segunda capa de max pooling con tamaño de pool 2x2.
- Una tercera capa convolucional con 64 filtros, tamaño de kernel 3x3 y activación ReLU.
- Una capa completamente conectada con 64 unidades y activación ReLU.
- Una capa de salida con 10 unidades y activación softmax (para una clasificación de 10 clases).



### Explicación

1. **Capas Convolucionales**:
   - `self.conv1`: 32 filtros, tamaño de kernel 3x3, activación ReLU.
   - `self.conv2`: 64 filtros, tamaño de kernel 3x3, activación ReLU.
   - `self.conv3`: 64 filtros, tamaño de kernel 3x3, activación ReLU.

2. **Capas de Pooling**:
   - `self.pool1`: Max pooling con tamaño 2x2.
   - `self.pool2`: Max pooling con tamaño 2x2.
   - `self.pool3`: Max pooling con tamaño 2x2.

3. **Capas Completamente Conectadas**:
   - `self.fc1`: 64 unidades, activación ReLU.
   - `self.fc2`: 10 unidades (para 10 clases de salida).

4. **Aplanado**:
   - `x = x.view(-1, 64 * 4 * 4)`: Aplanar las características antes de pasar a las capas completamente conectadas.

### Uso del Modelo

Para usar este modelo, puedes definir un conjunto de datos, un optimizador y un criterio de pérdida, y luego entrenar la red con tus datos.

In [4]:


import torch
import torch.nn as nn
import torch.nn.functional as F

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        # Capa convolucional 1: 32 filtros, kernel 3x3, activación ReLU
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1)
        # Capa de max pooling 1: tamaño del pool 2x2
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)

        # Capa convolucional 2: 64 filtros, kernel 3x3, activación ReLU
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        # Capa de max pooling 2: tamaño del pool 2x2
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)

        # Capa convolucional 3: 64 filtros, kernel 3x3, activación ReLU
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=1)
        # Capa de max pooling 3: tamaño del pool 2x2
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)
        
        # Capa completamente conectada: 64 unidades, activación ReLU
        self.fc1 = nn.Linear(64 * 4 * 4, 64)
        # Capa de salida: 10 unidades (para clasificación en 10 clases), activación softmax
        self.fc2 = nn.Linear(64, 10)

    def forward(self, x):
        # Aplicar la primera capa convolucional seguida de ReLU y pooling
        x = self.pool1(F.relu(self.conv1(x)))
        # Aplicar la segunda capa convolucional seguida de ReLU y pooling
        x = self.pool2(F.relu(self.conv2(x)))
        # Aplicar la tercera capa convolucional seguida de ReLU y pooling
        x = self.pool3(F.relu(self.conv3(x)))
        # Aplanar las características para la capa completamente conectada
        x = x.view(-1, 64 * 4 * 4)
        # Aplicar la primera capa completamente conectada seguida de ReLU
        x = F.relu(self.fc1(x))
        # Aplicar la capa de salida
        x = self.fc2(x)
        return x

# Crear una instancia de la red
net = CNN()

# Mostrar la arquitectura del modelo
print(net)


CNN(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=1024, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=10, bias=True)
)


# Descarga del modelo CIFAR-10
**Transformaciones para los datos de entrada**
Se define una serie de transformaciones que se aplicarán a las imágenes del conjunto de datos. En este caso, se convierte la imagen a un tensor y se normalizan los valores de los píxeles.

**Cargando el conjunto de entrenamiento**
Se descarga y carga el conjunto de datos CIFAR-10 para entrenamiento. Este conjunto contiene imágenes etiquetadas que se utilizarán para entrenar un modelo. Las imágenes se transforman según lo definido anteriormente.

**Cargando el conjunto de prueba**
Similar al conjunto de entrenamiento, se descarga y carga el conjunto de datos CIFAR-10 para pruebas. Este conjunto se utiliza para evaluar el rendimiento del modelo entrenado. Las imágenes también se transforman de la misma manera.

**Clases en CIFAR-10**
Se define una lista de las diez clases de objetos que están presentes en el conjunto de datos CIFAR-10, como aviones, automóviles, pájaros, gatos, etc.

In [5]:
import torchvision
import torchvision.transforms as transforms

# Transformaciones para los datos de entrada
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])  # Normalización para CIFAR-10

# Cargando el conjunto de entrenamiento
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

# Cargando el conjunto de prueba
testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

# Clases en CIFAR-10
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')


Files already downloaded and verified
Files already downloaded and verified


# Entrenamiento
**Función de pérdida y optimizador**
Se define la función que mide la discrepancia entre las predicciones del modelo y las etiquetas reales (en este caso, se usa la pérdida de entropía cruzada). También se define el optimizador, que ajusta los parámetros del modelo durante el entrenamiento para reducir la pérdida, utilizando el método de Gradiente Estocástico (SGD) con una tasa de aprendizaje y momento específicos.

**Ciclo de entrenamiento**
Se itera sobre el conjunto de datos varias veces (epochs). Para cada epoch:
- Se inicializa la pérdida acumulada.
- Se itera sobre los datos del conjunto de entrenamiento en lotes pequeños (mini-batches).
- Para cada lote:
  - Se obtienen las entradas y las etiquetas.
  - Se ponen a cero los gradientes de los parámetros del modelo.
  - Se calculan las predicciones del modelo y la pérdida.
  - Se realiza la retropropagación de la pérdida para calcular los gradientes.
  - Se actualizan los parámetros del modelo con el optimizador.
- Cada 2000 mini-batches, se imprime la pérdida media acumulada.

**Finalización del entrenamiento**
Se indica que el entrenamiento ha terminado.

In [6]:
import torch.optim as optim

# Función de pérdida y optimizador
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

# Ciclo de entrenamiento
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

print('Finished Training')

[1,  2000] loss: 2.151
[1,  4000] loss: 1.771
[1,  6000] loss: 1.560
[1,  8000] loss: 1.442
[1, 10000] loss: 1.338
[1, 12000] loss: 1.268
[2,  2000] loss: 1.158
[2,  4000] loss: 1.104
[2,  6000] loss: 1.059
[2,  8000] loss: 1.044
[2, 10000] loss: 0.990
[2, 12000] loss: 0.958
Finished Training
